# Main

## Input

### Import libraries

In [ ]:
import naas_python
import naas_data_product
from datetime import datetime, timedelta
import pytz
import yaml
import pydash
import os

### Setup variables

In [ ]:
config_path = os.path.join(naas_data_product.ROOT_PATH, 'config.yml')
config_secret_path = os.path.join(naas_data_product.ROOT_PATH, 'config.secret.yml')

## Model

### Function Helper

In [ ]:
def open_yaml_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = yaml.safe_load(file)
            return data
    except FileNotFoundError:
        print(f"Oops! The file at {file_path} was not found.")
    except yaml.YAMLError as exc:
        print(f"Error while parsing YAML file: {exc}")

### Get config variables

In [ ]:
workspace_id = None
storage_name = "abi"
access_key_id = None
secret_access_key = None
session_token = None
bucket_name = None
bucket_prefix = None

# Get data from YAML file
config_data = open_yaml_file(config_path) | open_yaml_file(config_secret_path)
if config_data:
    workspace_id = pydash.get(config_data, "config.workspace_id")
    storage_name = pydash.get(config_data, "config.storage_name")
    access_key_id = pydash.get(config_data, "storage.access_key_id")
    secret_access_key = pydash.get(config_data, "storage.secret_access_key")
    session_token = pydash.get(config_data, "storage.session_token")
    bucket_name = pydash.get(config_data, "storage.bucket_name")
    bucket_prefix = pydash.get(config_data, "storage.bucket_prefix")

### Get credentials

In [ ]:
def get_storage_credentials(
    workspace_id=None,
    storage_name=None,
):
    # Init
    if not workspace_id:
        api_key = naas_python.utils.domains_base.authorization.NaasSpaceAuthenticatorAdapter().jwt_token()
        workspace_id = get_personal_workspace(api_key)
#     print("Workspace ID:", workspace_id)
        
    # List storage
    result = naas_python.storage.list_workspace_storage(workspace_id=workspace_id)
    storages = result.get("storage")
    storage_exist = False
    for storage in storages:
        if storage.get("name") == storage_name:
            storage_exist = True
            new_storage = storage
            
    # Create storage
    if not storage_exist:
        new_storage = naas_python.storage.create_workspace_storage(workspace_id=workspace_id, storage_name=storage_name).get("storage")
        
    # Get storage credentials
    credentials = naas_python.storage.create_workspace_storage_credentials(workspace_id=workspace_id, storage_name=storage_name)
    return credentials, workspace_id

if not access_key_id and not secret_access_key:
    credentials, workspace_id = get_storage_credentials(workspace_id=workspace_id, storage_name=storage_name)
    if len(credentials) > 0:
        access_key_id = pydash.get(credentials, "credentials.s3.access_key_id")
        secret_access_key = pydash.get(credentials, "credentials.s3.secret_key")
        session_token = pydash.get(credentials, "credentials.s3.session_token")
        endpoint_url = pydash.get(credentials, "credentials.s3.endpoint_url")
        bucket_name = endpoint_url.split("s3://")[1].split("/")[0]
        bucket_prefix = endpoint_url.split(f"{bucket_name}/")[1]
        region_name = pydash.get(credentials, "credentials.s3.region_name")

## Output

### Storage Manager

In [ ]:
sm = StorageManager(access_key_id, secret_access_key, session_token, bucket_name, bucket_prefix)

### Asset Manager

In [ ]:
am = AssetManager(workspace_id, storage_name, sm)

### Timezone

In [ ]:
# Timezone
tz = sm.pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "timezone")
if tz is None:
    tz = "Europe/Paris"
TIMEZONE = pytz.timezone(tz)

### Scenarios

In [ ]:
TW = datetime.now(TIMEZONE).strftime("W%W-%Y")
LW = (datetime.now(TIMEZONE) - timedelta(days=datetime.now(TIMEZONE).weekday() + 7)).strftime("W%W-%Y")

# Mapping colors
MAPPING_COLORS = {
    TW: "#48DD82",
    LW: "#FFFDA2",
}

### Logos

In [ ]:
# Arrows
arrow_up = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Eo_circle_green_arrow-up.svg/2048px-Eo_circle_green_arrow-up.svg.png"
arrow_down = "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Eo_circle_red_arrow-down.svg/2048px-Eo_circle_red_arrow-down.svg.png"
arrow_right = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Eo_circle_orange_arrow-right.svg/2048px-Eo_circle_orange_arrow-right.svg.png"